## Reddit 댓글 규칙 위반 여부 분류

In [13]:
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVR

In [14]:
!pip install -q kaggle
!pip install xgboost
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.7 MB/s eta 0:00:00a 0:00:01


In [15]:
os.environ['KAGGLE_USERNAME'] = "yuminlee72"
os.environ['KAGGLE_KEY'] = "1e586eaad50541a8cc03a63cf720dc79"

In [16]:
!kaggle competitions download -c jigsaw-agile-community-rules

jigsaw-agile-community-rules.zip: Skipping, found more recently modified local copy (use --force to force download)


In [17]:
!ls -alh

total 1416
drwxr-xr-x  10 iyumin  staff   320B 10 11 14:45 .
drwxr-xr-x@  7 iyumin  staff   224B 10  2 01:16 ..
drwxr-xr-x  10 iyumin  staff   320B 10  9 11:10 .git
drwxr-xr-x   3 iyumin  staff    96B 10 10 14:04 .ipynb_checkpoints
drwxr-xr-x   3 iyumin  staff    96B 10 10 13:14 .virtual_documents
-rw-r--r--@  1 iyumin  staff    11B 10  9 11:09 01.py
drwxr-xr-x   3 iyumin  staff    96B 10 10 13:13 anaconda_projects
drwxr-xr-x@  3 iyumin  staff    96B 10 10 21:17 data
-rw-r--r--@  1 iyumin  staff   695K  7 21 15:52 jigsaw-agile-community-rules.zip
-rw-r--r--   1 iyumin  staff   7.4K 10 11 14:45 project01.ipynb


In [18]:
!unzip -l jigsaw-agile-community-rules.zip

Archive:  jigsaw-agile-community-rules.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
      112  07-21-2025 15:52   sample_submission.csv
     9167  07-21-2025 15:52   test.csv
  1971422  07-21-2025 15:52   train.csv
---------                     -------
  1980701                     3 files


In [19]:
# 새 폴더 만들기 / -p: 중간 경로가 없어도 부모 폴더까지 같이 생성 (이미 있어도 에러 없이 넘어감)
!mkdir -p data/jigsaw_agile

# 압축 풀기 / -q: 조용히 진행, -o(overwrite): 같은 이름의 파일이 있어도 덮어씀
# -d: 풀린 파일을 저장할 목적지 폴더 지정
!unzip -q -o jigsaw-agile-community-rules.zip -d data/jigsaw_agile

# 폴더 내용 보기 / -a: 숨김 파일까지 모두 표시, -l: 자세히 표시, -h: 파일을 KB/MB 단위로 표시
!ls -alh data/jigsaw_agile

total 3888
drwxr-xr-x@ 5 iyumin  staff   160B 10 11 14:47 .
drwxr-xr-x@ 3 iyumin  staff    96B 10 10 21:17 ..
-rw-r--r--@ 1 iyumin  staff   112B  7 21 15:52 sample_submission.csv
-rw-r--r--@ 1 iyumin  staff   9.0K  7 21 15:52 test.csv
-rw-r--r--@ 1 iyumin  staff   1.9M  7 21 15:52 train.csv


In [20]:
train_df = pd.read_csv('data/jigsaw_agile/train.csv')
test_df = pd.read_csv('data/jigsaw_agile/test.csv')

train_df.head()

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1
